In [3]:
#!pip install textblob
import nltk
from nltk.tokenize import word_tokenize  
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

In [4]:
import pandas as pd
df = pd.read_csv('Data/ms4h05_data_anon.csv')
print(df.to_string()) 

      status_id                                                                                                                                                                                                                                                                                                                                                                                                       text                       annontation
0             0                                                                                                                                                                                                                                                                      PAGS PARODY! Plus, Joe talks coronavirus and has special guest Peter J Hasson on his new book. - @JoeTalkShow https://t.co/IJE4gZwI0V                        irrelevant
1             1                                                                                         #coron

### Task A:  Pre-Processing

#### 1. Change @twitterhandle to @user

In [5]:
df["text"][0] #example of a tweet with @username

'PAGS PARODY! Plus, Joe talks coronavirus and has special guest Peter J Hasson on his new book. - @JoeTalkShow https://t.co/IJE4gZwI0V'

In [6]:
import re
df['text'] = [re.sub(r'@\w+', '@User', x) for x in df["text"]] # eliminate usernames and replace with @User

In [7]:
df['text'][0] #above changes @JoeTalkShow to @User

'PAGS PARODY! Plus, Joe talks coronavirus and has special guest Peter J Hasson on his new book. - @User https://t.co/IJE4gZwI0V'

#### 2. Clean tweets

In [8]:
import html

def clean(text):
    # convert html escapes like &amp; to characters.
    text = html.unescape(text)
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    #text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)
    # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [9]:
df['clean_text'] = df['text'].map(clean)

In [10]:
df['text'][352]

"Hey @User if you are done with cocaine and gay orgies, take some similar diplomatic initiatives..\n\nStop being Bajwa's bitch for once and show your balls, if you have any..\n\n#imrankhanPTI #Pakistan #CoronaAlert \n#CoronaOutbreak #CoronavirusPandemic #CoronaVirus #SAARC https://t.co/55n0bN9ZRv"

In [11]:
df['clean_text'][352]

"Hey @User if you are done with cocaine and gay orgies, take some similar diplomatic initiatives.. Stop being Bajwa's bitch for once and show your balls, if you have any.. #imrankhanPTI #Pakistan #CoronaAlert #CoronaOutbreak #CoronavirusPandemic #CoronaVirus #SAARC https://t.co/55n0bN9ZRv"

#### 3. Remove specified hashtags

In [12]:
#store stated hashtags
remove_words = ['#nCoV20199',
'#CoronaOutbreak',
'#CoronaVirus',
'#CoronavirusCoverup',
'#CoronavirusOutbreak',
'#COVID19',
'#Coronavirus',
'#WuhanCoronavirus',
'#coronaviris',
'#Wuhan']

In [13]:
import re
p = re.compile('|'.join(map(re.escape, remove_words)))
df['ctext_nh'] = [p.sub('', text) for text in df['clean_text']] 

In [14]:
df['clean_text'][352] #tweet with coronaoutbreak

"Hey @User if you are done with cocaine and gay orgies, take some similar diplomatic initiatives.. Stop being Bajwa's bitch for once and show your balls, if you have any.. #imrankhanPTI #Pakistan #CoronaAlert #CoronaOutbreak #CoronavirusPandemic #CoronaVirus #SAARC https://t.co/55n0bN9ZRv"

In [15]:
df['ctext_nh'][352] #tweet where it should be removed 

"Hey @User if you are done with cocaine and gay orgies, take some similar diplomatic initiatives.. Stop being Bajwa's bitch for once and show your balls, if you have any.. #imrankhanPTI #Pakistan #CoronaAlert  Pandemic  #SAARC https://t.co/55n0bN9ZRv"

#### 4. Convert all tweets to lowercase for tokenisation

In [16]:
df = df.applymap(lambda s:s.lower() if type(s) == str else s) #convert all to lowercase

#### 5. Get rid of punctuation

In [17]:
def form_sentence(tweet):
    tweet_blob = TextBlob(tweet)
    return ' '.join(tweet_blob.words)

#this gives an example what it's doing, comparing tweet after and tweet before
print(form_sentence(df['clean_text'].iloc[1]))
print(df['clean_text'].iloc[1])


coronavirus shows that racists have absolutely 2 much time on their hands firstly those racists who hate poc r stuck in xenophobicworld are proud deplorables which means they did n't even beleev coronavirus existed until a day or 2 ago when trump did they 're quick studies https t.co/p22j3ti6kk
#coronavirus shows that racists have absolutely 2 much time on their hands firstly, those racists who hate poc r stuck in #xenophobicworld are proud #deplorables which means they didn't even beleev #coronavirus existed until a day or 2 ago when trump did they're quick studies https://t.co/p22j3ti6kk


In [18]:
df['text5'] = df['ctext_nh'].map(form_sentence)

In [19]:
df['text5']

0       pags parody plus joe talks coronavirus and has...
1       coronavirus shows that racists have absolutely...
2       user user i would expect front line staff to b...
3       the coronavirus is not only affecting the way ...
4       i hope your father recovers from the virus as ...
                              ...                        
3637    da fark is wrong with these people https t.co/...
3638    doctor finds cure to covid-19 without vaccine ...
3639    i can ’ t believe people legitimately think th...
3640    wrong with trump continues will push pressure ...
3641    for anyone who still believes that covid-19 is...
Name: text5, Length: 3642, dtype: object

#### 6. Get rid of stopwords and also use tokens and bigrams

In [20]:
df['text5'][1]

"coronavirus shows that racists have absolutely 2 much time on their hands firstly those racists who hate poc r stuck in xenophobicworld are proud deplorables which means they did n't even beleev coronavirus existed until a day or 2 ago when trump did they 're quick studies https t.co/p22j3ti6kk"

In [21]:
def no_user_alpha(tweet):
    tweet_list = [ele for ele in tweet.split() if ele != 'user']
    clean_tokens = [t for t in tweet_list if re.match(r'[^\W\d]*$', t)]
    clean_s = ' '.join(clean_tokens)
    msg_split =clean_s.split()
    grams = list(everygrams(msg_split, max_len=2))
    result = list(map(join_tuple_string, grams))
    clean_mess = [word for word in result if word.lower() not in stopwords.words('english')]
    return clean_mess
#example of 2 tweets, one after the code applied, one after
print(no_user_alpha(form_sentence((df['ctext_nh'].iloc[1]))))
print(df['clean_text'].iloc[1])

NameError: name 'everygrams' is not defined

In [ ]:
df['text6'] = df['text5'].map(no_user_alpha)

#### 7. Normalisation (Lemmatisation)

In [ ]:
def normalisation(tweet_list):
        lem = WordNetLemmatizer()
        normalized_tweet = []
        for word in tweet_list:
            normalized_text = lem.lemmatize(word,'v')
            normalized_tweet.append(normalized_text)
        return normalized_tweet
    

print(normalisation(no_user_alpha(form_sentence((df['clean_text'].iloc[1])))))

In [ ]:
df['text7'] = df['text6'].map(normalisation)
df['text7']

### Task B